# Week 4 – Simple ETL in Azure Databricks


In [ ]:
!pip install pyspark --quiet

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, trim, upper, datediff, current_date

In [ ]:
# Install dependencies (only needed in Google Colab)
# In Azure Databricks, skip this cell
!pip install pyspark==3.5.1 delta-spark==3.1.0
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

  Using cached delta_spark-3.1.0-py3-none-any.whl.metadata (1.9 kB)
Using cached delta_spark-3.1.0-py3-none-any.whl (21 kB)
  Attempting uninstall: delta-spark
    Found existing installation: delta-spark 3.2.0
    Uninstalling delta-spark-3.2.0:
      Successfully uninstalled delta-spark-3.2.0


In [ ]:
# using DELTA in colab
!pip install delta-spark==3.2.0 -q
import pyspark
from delta import *
from pyspark.sql.functions import *

# Create a SparkSession with Delta Lake extensions
# The '.config(...)' lines are crucial for enabling Delta Lake's features
builder = pyspark.sql.SparkSession.builder.appName("DeltaDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Get or create the SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark and Delta Lake are ready!")


Spark and Delta Lake are ready!


# Upload CSV data into Databricks

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload orders_table.csv, suppliers_table.csv, inventory_table.csv


Saving inventory_table.csv to inventory_table (1).csv
Saving orders_table.csv to orders_table (1).csv
Saving suppliers_table.csv to suppliers_table (1).csv


In [ ]:
orders_df = spark.read.csv("orders_table.csv", header=True, inferSchema=True)
suppliers_df = spark.read.csv("suppliers_table.csv", header=True, inferSchema=True)
inventory_df = spark.read.csv("inventory_table.csv", header=True, inferSchema=True)

orders_df.show(5)


+--------+-----------+------------+----------+-------------+---------+--------+
|order_id|supplier_id|inventory_id|order_date|delivery_date|   status|quantity|
+--------+-----------+------------+----------+-------------+---------+--------+
|       1|          1|           1|2025-07-01|   2025-07-03|Delivered|      30|
|       2|          2|           2|2025-07-05|   2025-07-23|Delivered|      10|
|       3|          2|           3|2025-07-06|   2025-07-09|Delivered|      20|
|       4|          3|           4|2025-07-10|         NULL|  Shipped|       5|
|       5|          1|           5|2025-07-12|         NULL|  Pending|      12|
+--------+-----------+------------+----------+-------------+---------+--------+
only showing top 5 rows



# Run a notebook to clean and filter the data

In [ ]:
from pyspark.sql.functions import col, to_date, trim, upper

# Step 1: Clean the data (convert dates, trim text, uppercase status)
orders_df = (orders_df
    .withColumn("order_date", to_date(col("order_date")))
    .withColumn("delivery_date", to_date(col("delivery_date")))
    .withColumn("status", upper(trim(col("status"))))
)

# Show cleaned orders table
print("Cleaned Orders Table:")
orders_df.show(10, truncate=False)

# Step 2: Filter only DELIVERED orders
delivered_df = orders_df.filter(col("status") == "DELIVERED")

print("Filtered DELIVERED Orders:")
delivered_df.show(10, truncate=False)


Cleaned Orders Table:
+--------+-----------+------------+----------+-------------+---------+--------+
|order_id|supplier_id|inventory_id|order_date|delivery_date|status   |quantity|
+--------+-----------+------------+----------+-------------+---------+--------+
|1       |1          |1           |2025-07-01|2025-07-03   |DELIVERED|30      |
|2       |2          |2           |2025-07-05|2025-07-23   |DELIVERED|10      |
|3       |2          |3           |2025-07-06|2025-07-09   |DELIVERED|20      |
|4       |3          |4           |2025-07-10|NULL         |SHIPPED  |5       |
|5       |1          |5           |2025-07-12|NULL         |PENDING  |12      |
|6       |2          |3           |2025-07-20|NULL         |PENDING  |20      |
|7       |1          |5           |2025-07-20|NULL         |PENDING  |20      |
|9       |2          |3           |2025-07-23|NULL         |PENDING  |20      |
|10      |1          |5           |2025-07-23|NULL         |PENDING  |20      |
+--------+--------

# Save cleaned output as Delta or CSV

In [ ]:
# 3. Save cleaned output as Delta or CSV
orders_df.write.format("delta").mode("overwrite").save("/FileStore/tables/orders_cleaned_delta")
orders_df.write.format("csv").option("header", "true").mode("overwrite").save("/FileStore/tables/orders_cleaned_csv")


# Run basic analysis queries using SQL or PySpark

# Run basic analysis queries using PySpark

1. Average delivery time in days

In [ ]:
from pyspark.sql import functions as F

orders_df.withColumn(
    "delivery_days", F.datediff("delivery_date", "order_date")
).agg(F.avg("delivery_days").alias("avg_delivery_days")).show()

+-----------------+
|avg_delivery_days|
+-----------------+
|7.666666666666667|
+-----------------+



2. Supplier with highest total quantity ordered

In [ ]:

orders_df.groupBy("supplier_id") \
    .agg(F.sum("quantity").alias("total_quantity")) \
    .orderBy(F.desc("total_quantity")) \
    .show(1)

+-----------+--------------+
|supplier_id|total_quantity|
+-----------+--------------+
|          1|            82|
+-----------+--------------+
only showing top 1 row



3. Percentage of delivered orders

In [ ]:
total_orders = orders_df.count()
delivered_orders = orders_df.filter(F.col("status") == "DELIVERED").count()
delivery_percentage = (delivered_orders / total_orders) * 100
print(f"Delivery Percentage: {delivery_percentage:.2f}%")

Delivery Percentage: 33.33%


4. Find the average quantity per order

In [ ]:
orders_df.agg(F.avg("quantity").alias("avg_quantity")).show()

+------------------+
|      avg_quantity|
+------------------+
|17.444444444444443|
+------------------+



5. Top 3 suppliers with most orders

In [ ]:
orders_df.groupBy("supplier_id").count().orderBy(F.desc("count")).show(3)

+-----------+-----+
|supplier_id|count|
+-----------+-----+
|          1|    4|
|          2|    4|
|          3|    1|
+-----------+-----+



# Run basic analysis queries using SQL

In [ ]:
orders_df.createOrReplaceTempView("orders")

1. Top 3 suppliers by average order quantity

In [ ]:
spark.sql("""
    SELECT supplier_id, AVG(quantity) AS avg_quantity
    FROM orders
    GROUP BY supplier_id
    ORDER BY avg_quantity DESC
    LIMIT 3
""").show()

+-----------+------------+
|supplier_id|avg_quantity|
+-----------+------------+
|          1|        20.5|
|          2|        17.5|
|          3|         5.0|
+-----------+------------+



2. Number of orders per status

In [ ]:
spark.sql("""
    SELECT status, COUNT(*) AS total_orders
    FROM orders
    GROUP BY status
    ORDER BY total_orders DESC
""").show()

+---------+------------+
|   status|total_orders|
+---------+------------+
|  PENDING|           5|
|DELIVERED|           3|
|  SHIPPED|           1|
+---------+------------+



3. Orders delayed more than 5 days

In [ ]:
spark.sql("""
    SELECT order_id, supplier_id, DATEDIFF(delivery_date, order_date) AS delay_days
    FROM orders
    WHERE DATEDIFF(delivery_date, order_date) > 5
    ORDER BY delay_days DESC
""").show()

+--------+-----------+----------+
|order_id|supplier_id|delay_days|
+--------+-----------+----------+
|       2|          2|        18|
+--------+-----------+----------+



4. Earliest and latest delivery date per supplier

In [ ]:
spark.sql("""
    SELECT supplier_id,
           MIN(delivery_date) AS earliest_delivery,
           MAX(delivery_date) AS latest_delivery
    FROM orders
    GROUP BY supplier_id
""").show()

+-----------+-----------------+---------------+
|supplier_id|earliest_delivery|latest_delivery|
+-----------+-----------------+---------------+
|          1|       2025-07-03|     2025-07-03|
|          3|             NULL|           NULL|
|          2|       2025-07-09|     2025-07-23|
+-----------+-----------------+---------------+



# Deliverables:

# Cleaned output stored in CSV format

In [ ]:
# Save cleaned DataFrame as a single CSV file
clean_csv_path = "/content/orders_cleaned.csv"



In [ ]:
orders_df.coalesce(1) \
    .write.option("header", "true") \
    .mode("overwrite") \
    .csv("/content/orders_cleaned_single")

In [ ]:
import shutil, glob
part_file = glob.glob("/content/orders_cleaned_single/part-*.csv")[0]
shutil.move(part_file, clean_csv_path)

print(f"Cleaned CSV saved at: {clean_csv_path}")


Cleaned CSV saved at: /content/orders_cleaned.csv


In [ ]:
# Download the CSV
from google.colab import files
files.download(clean_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>